# Parámetros

In [1]:
year = 2022
week = 8
#path = r'C:\Users\jshernandezm\OneDrive - genommalabinternacional\Info_POS_clientes\Sell Out - No B2b\Sumed'
path = r'C:\Users\amartinezgo\genommalabinternacional\Francisco Jose Delfino - Info_POS_clientes\Sell Out - No B2b\Sumed'

# Libraries

In [2]:
import pandas as pd
from datetime import datetime
import pyodbc
import shutil
import os
from openpyxl import load_workbook

# Conexiones y querys

In [3]:
conn1 = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_DWH;'
                     'Trusted_Connection=yes;')

conn2 = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_MasterOp;'
                     'Trusted_Connection=yes;')

query_dates = '''
                SELECT TmpFecha
                      ,TmpSemanaAnioGenomma
                      ,TmpAnioSemanaGenomma

                  FROM dbo.Dim_Tiempo where TmpDiaSemana = 7 order by TmpID
'''

query_nombres = '''
                SELECT DISTINCT ProPstCodBarras, ProPstNombre
                  FROM dbo.vw_EstructuraProductosTotalPaises
                  WHERE País = 'Argentina';
'''

# Importar Datos

In [4]:
filename = '{0}_{1}_Sumed_SO.xlsx'.format(year, str(week).zfill(2))
file_names = [filename]

In [5]:
#Read SQL and drop duplicates
nombres = pd.read_sql(query_nombres,conn2)
nombres.drop_duplicates(subset=['ProPstCodBarras'],inplace=True)
nombres.columns = ['EAN o UPC Genomma','Desc']

In [6]:
dates = pd.read_sql(query_dates,conn1)
df_all = pd.DataFrame()
files_to_check = []

for file in file_names:
    #Save year and week from file name
    year = int(file.split('_')[0])
    week = int(file.split('_')[1])

    df = pd.read_excel(path + '\\' + file, header=1)
    #Check if the file has 3 or 4 columns
    if len(list(df.columns)) == 4:
        df.columns = ['EAN','DESC','ALIAS','CANTIDAD']
    else:
        df.columns = ['EAN','DESC','CANTIDAD']
    #Drop last row if it has the total units
    if df.loc[df.iloc[-1:,-1:].index,'CANTIDAD'].values == df.iloc[:-1]['CANTIDAD'].sum():
        df.drop(df.iloc[-1:,-1:].index,inplace=True)

    df.fillna(value={'EAN':0},inplace=True)

    data_final = pd.DataFrame()
    #Set values for distinct columns
    data_final['Unidades'] = df['CANTIDAD']
    data_final['Fecha_Venta'] = dates[(dates['TmpAnioSemanaGenomma'] == year) & (dates['TmpSemanaAnioGenomma'] == week)]['TmpFecha'].unique()[0]
    data_final['Fecha_Venta'] = data_final['Fecha_Venta'].dt.date
    data_final['Cadena'] = 'Sumed'
    data_final['Cod_Prod de la Cadena'] = df['EAN'].astype('int64')
    data_final['EAN o UPC Genomma'] = df['EAN'].astype('int64')
    data_final['Descripcion_Prod'] = df['DESC']
    #Store code manually inserted
    data_final['Cod_Local'] = '1'
    data_final['Descripcion_Local'] = 'Sumed Local Generico'
    data_final = pd.merge(data_final,nombres, on='EAN o UPC Genomma', how='left')
    #Rearrange dataframe and save it
    #data_final = data_final[['Fecha_Venta','Cadena','Cod_Prod de la Cadena','EAN o UPC Genomma','Desc','Cod_Local','Descripcion_Local','Unidades']]
    data_final = data_final[['Fecha_Venta','Cadena','Cod_Prod de la Cadena','EAN o UPC Genomma','Descripcion_Prod','Cod_Local','Descripcion_Local','Unidades']]
    data_final.dropna(subset=['Unidades'],inplace=True)
    #data_final['Desc'] = data_final['Desc'].apply(lambda x: x.strip())
    data_final['Descripcion_Local'] = data_final['Descripcion_Local'].apply(lambda x: x.strip())
    df_all = pd.concat([df_all,data_final])
    print('Listo archivo ' + file)
        

df_all.reset_index(drop=True,inplace=True)
#Check for missing EAN
if df_all[df_all['EAN o UPC Genomma']==0].shape[0] > 0:
    print('Revisar archivo, producto sin EAN...')

#CODIGO DE ABAJO NO SIRVE SI SE CORRE UNA SEMANA ES PARA LLENAR LOS EAN DEL HISTORICO...
# if df_all[df_all['EAN o UPC Genomma'] == 0].shape[0] > 0:
#     df_all['key'] = df_all['EAN o UPC Genomma'].astype(str) + '-' + df_all['Descripcion_Prod'].astype(str) 
#     df_aux = df_all[df_all['EAN o UPC Genomma']==0].copy()
#     df_aux['EAN o UPC Genomma'] = df_aux['Descripcion_Prod'].apply(lambda x: int(df_all[df_all['Descripcion_Prod'] == x]['key'].value_counts().index[0].split('-')[0]) if int(df_all[df_all['Descripcion_Prod'] == x]['key'].value_counts().index[0].split('-')[0]) != 0 else int(df_all[df_all['Descripcion_Prod'] == x]['key'].value_counts().index[2].split('-')[0]))
#     df_all.drop(df_aux.index,axis=0,inplace=True)
#     df_all = pd.concat([df_all,df_aux])

# df_all = pd.merge(df_all,nombres, on='EAN o UPC Genomma', how='left')
# df_all['Cod_Prod de la Cadena'] = df_all['EAN o UPC Genomma']
# df_all =  df_all[['Fecha_Venta','Cadena','Cod_Prod de la Cadena','EAN o UPC Genomma','Descripcion_Prod','Desc','Cod_Local','Descripcion_Local','Unidades']]
    

Listo archivo 2022_08_Sumed_SO.xlsx


In [7]:
#Check if if there's a columns with missing values
df_all.isnull().sum()

Fecha_Venta              0
Cadena                   0
Cod_Prod de la Cadena    0
EAN o UPC Genomma        0
Descripcion_Prod         0
Cod_Local                0
Descripcion_Local        0
Unidades                 0
dtype: int64

# Exportación

In [8]:
def write_layout(filename, df, sellout=True):
    writer = pd.ExcelWriter(filename, engine='openpyxl', mode='a')
    writer.book = load_workbook(filename)
    writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    if sellout == True:
        df.to_excel(writer, sheet_name='Ventas', startcol=1, startrow=18, index=False, header=None)
    else:
        df.to_excel(writer, sheet_name='Stock', startcol=1, startrow=18, index=False, header=None)
    writer.save()

Toma el layout de ISV y crea la carpeta correspondiente:

In [9]:
original = '../../../../1Data/2Catalogue/toISV/Layout/Formato No B2B Orig.xlsx'
new_folder = '../../../../1Data/2Catalogue/toISV/{0}/S{1}/'.format(year, str(week).zfill(2))

if not os.path.exists(new_folder):
    os.mkdir(new_folder)

In [10]:
%%time
filename = 'Formato No B2B SUMED Sem ' + str(week).zfill(2)
path_export = new_folder + '/' + filename +'.xlsx'
shutil.copy(original, path_export)
write_layout(path_export, df_all, True)

Wall time: 2.44 s
